# Fine-tune LLMs to do Sarcasm Detections and Interpretations

In [1]:
!pip install nltk comet-ml emoji unbabel-comet datasets evaluate rouge_score

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.1/698.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set the target directory path
target_dir = '/content/drive/MyDrive/SarcasmNLP'

# Create the directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Change the working directory to the target directory
os.chdir(target_dir)

print(f"Current working directory: {os.getcwd()}")

Mounted at /content/drive
Current working directory: /content/drive/MyDrive/SarcasmNLP


In [3]:
model_choice = 'gpt2'
#model_choice = 'flan-t5-base'
# model_choice = 't5-base'
classifier_model_choice = 'bert-base-uncased'


In [4]:
mode = 'train'
# mode = 'evaluate'

In [5]:
#dataset_ = 'iSarcasm'
# dataset_ = 'GPT-4o-mini'
dataset_ = 'combined_train_df'

## Load Model

### Classfication Model: bert-base-uncased

In [6]:
# initialize tokenizer and model for Sarcasm Detection
from transformers import BertTokenizer, BertForSequenceClassification

if mode == 'train':
  classifier_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
  classifier_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
else:
  classifier_tokenizer = BertTokenizer.from_pretrained(f'./results/{classifier_model_choice}/my_model')
  classifier_model = BertForSequenceClassification.from_pretrained(f'./results/{classifier_model_choice}/my_model')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### GPT-2 small

In [7]:
if model_choice == 'gpt2':
  from transformers import GPT2Tokenizer, GPT2LMHeadModel
  if mode == 'train':
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
  else:
    tokenizer = GPT2Tokenizer.from_pretrained(f'./results/{model_choice}/my_model')
    model = GPT2LMHeadModel.from_pretrained(f'./results/{model_choice}/my_model')


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

### Google FLAN-T5-base

In [8]:
if model_choice == 'flan-t5-base':
  from transformers import T5Tokenizer, T5ForConditionalGeneration
  if mode == 'train':
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
  else:
    tokenizer = T5Tokenizer.from_pretrained(f'./results/{model_choice}/my_model')
    model = T5ForConditionalGeneration.from_pretrained(f'./results/{model_choice}/my_model')


### T5-base

In [9]:
if model_choice == 't5-base':
  from transformers import T5Tokenizer, T5ForConditionalGeneration
  if mode == 'train':
    tokenizer = T5Tokenizer.from_pretrained("t5-base")
    model = T5ForConditionalGeneration.from_pretrained("t5-base")
  else:
    tokenizer = T5Tokenizer.from_pretrained(f'./results/{model_choice}/my_model')
    model = T5ForConditionalGeneration.from_pretrained(f'./results/{model_choice}/my_model')


## Load Data

In [10]:
# loading combined_df dataset = isarcasm + gpt_pairs
import pandas as pd
def load_data():
  dataset = pd.read_csv('combined_df.tsv', sep='\t')
  evaluation_dataset = pd.read_csv('iSarcasm_pairs_test.tsv', sep='\t')
  dataset['Translation'] = dataset['Translation'].fillna('')
  evaluation_dataset['Translation'] = evaluation_dataset['Translation'].fillna('')
  return dataset, evaluation_dataset

df, df_eval = load_data()

In [11]:
df.head()

,Sarcastic,Translation,IsSarcastic
0,I deeply regret downloading tiktok.....yet not...,,0
1,It's been so lovely to see the world in blue a...,,0
2,quackity stream is making me so genuinely happ...,,0
3,"Oh yeah, because staying up late is the best d...",Staying up late is not a good choice for my he...,1
4,Fantastic! I can't wait for the meeting just t...,"Repeated discussions can feel tedious.""",1


## Classification model:

### Intitialization

In [12]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [14]:
list(test_df['IsSarcastic']).count(0)

248

In [15]:
# Function to convert emojis to text, handling float values
import emoji

def convert_emojis(text):
    # Check if text is a float (potentially NaN) and convert to string
    if isinstance(text, float):
        text = str(text)
    return emoji.demojize(text, delimiters=(" ", " "))

# Apply emoji conversion to both input (sarcastic) and output (literal) text
train_df['Sarcastic'] = train_df['Sarcastic'].apply(convert_emojis)
valid_df['Sarcastic'] = valid_df['Sarcastic'].apply(convert_emojis)
test_df['Sarcastic'] = test_df['Sarcastic'].apply(convert_emojis)

In [16]:
# source text encoding with selecetd classifier tokenizer
def encode_texts(texts, targets=None, tokenizer=None):
    if targets is None:  # For single input (sarcasm detection)
        return classifier_tokenizer(
            texts.tolist(),
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

In [17]:
import torch
from torch.utils.data import Dataset

# Custom dataset class for sarcasm classification
class SarcasmClassificationDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = encode_texts(texts.astype(str))
        self.labels = torch.tensor(labels.values)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'input_ids': self.texts['input_ids'][idx],
            'attention_mask': self.texts['attention_mask'][idx],
            'labels': self.labels[idx]
        }

# Create datasets
train_dataset = SarcasmClassificationDataset(train_df['Sarcastic'], train_df['IsSarcastic'])
valid_dataset = SarcasmClassificationDataset(valid_df['Sarcastic'], valid_df['IsSarcastic'])
test_dataset = SarcasmClassificationDataset(test_df['Sarcastic'], test_df['IsSarcastic'])

### Training of Sarcasm Detection model

In [18]:
# metric computation for sarcasm detection
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

def compute_metrics(pred):
    # Extract predictions and labels
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    # Calculate metrics
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')

    # Return as dictionary
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [19]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Training args for classification/detection model
training_args = TrainingArguments(
    output_dir=f'./results/{classifier_model_choice}',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_dir='./logs',
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    weight_decay=0.01,
    learning_rate=9e-5,
    save_strategy="epoch"
)

# Initialize Trainer for sarcasm classification
trainer = Trainer(
    model=classifier_model.to(device),
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    compute_metrics=compute_metrics       # Metrics function
)

In [20]:
if mode == 'train':
  # Train the sarcasm classifier
  trainer.train()
  # Save the model
  classifier_model.save_pretrained(f'./results/{classifier_model_choice}/my_model')
  classifier_tokenizer.save_pretrained(f'./results/{classifier_model_choice}/my_model')

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.402400,0.386159,0.841828,0.975309,0.738318,0.840426
2,0.361400,0.363549,0.847100,0.979508,0.744548,0.846018
3,0.344800,0.335381,0.855888,0.979920,0.760125,0.856140
4,0.285300,0.542238,0.845343,0.894915,0.822430,0.857143


In [21]:
pred = trainer.predict(test_dataset)

In [22]:
compute_metrics(pred)

{'accuracy': 0.8558875219683656,
 'precision': 0.9799196787148594,
 'recall': 0.7601246105919003,
 'f1': 0.856140350877193}

In [23]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.335380882024765, 'eval_accuracy': 0.8558875219683656, 'eval_precision': 0.9799196787148594, 'eval_recall': 0.7601246105919003, 'eval_f1': 0.856140350877193, 'eval_runtime': 2.8492, 'eval_samples_per_second': 199.704, 'eval_steps_per_second': 25.27, 'epoch': 4.0}


## Interpretation Model:

### Initialization

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Data loading for Interpretation model, only using the sarcastic statements
df, df_eval = load_data()
df = df[df['IsSarcastic'] == 1]
print(df.shape)

(3130, 3)


In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
add_prefix = lambda x: "Provide straightforward, literal translations for this sarcastic comment: " + x

train_df['Input'] = train_df['Sarcastic'].apply(add_prefix)
valid_df['Input'] = valid_df['Sarcastic'].apply(add_prefix)
test_df['Input'] = test_df['Sarcastic'].apply(add_prefix)


In [ ]:
tokenizer.pad_token = tokenizer.eos_token


def tokenize_data(df):
    inputs = tokenizer(
        df['Input'].tolist(),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    targets = tokenizer(
        df['Translation'].tolist(),
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )

    # Set padding tokens in targets to -100 to ignore them in loss calculation
    targets['input_ids'][targets['input_ids'] == tokenizer.pad_token_id] = -100

    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': targets['input_ids'],
    }

# Tokenize train, validation, and test datasets
train_encodings = tokenize_data(train_df)
valid_encodings = tokenize_data(valid_df)
test_encodings = tokenize_data(test_df)

In [ ]:
import torch

class SarcasmTranslationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Create datasets
train_dataset = SarcasmTranslationDataset(train_encodings)
valid_dataset = SarcasmTranslationDataset(valid_encodings)
test_dataset = SarcasmTranslationDataset(test_encodings)

### Prepare Metrics

For colab, need to install additional packages (already in conda environment.yml)

In [ ]:
import evaluate

# Load the metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
comet = evaluate.load("comet")  # Ensure COMET is installed and properly configured
chrf = evaluate.load("chrf")  # ChrF metric



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../root/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:

def compute_metrics(pred):
    # Get predictions and labels
    predictions = pred.predictions[0]
    labels = pred.label_ids

    # Decode predictions and labels
    # Ensure predictions are flattened and contain token IDs:
    predictions = predictions.argmax(-1)  # Assuming predictions are logits

    if isinstance(predictions[0], list):
        predictions = [item for sublist in predictions for item in sublist]

    if isinstance(labels[0], list):
        labels = [item for sublist in labels for item in sublist]


    # Decode predictions and labels
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # BLEU
    bleu_result = bleu.compute(predictions=decoded_preds, references=decoded_labels)

    # ChrF
    chrf_result = chrf.compute(predictions=decoded_preds, references=decoded_labels)

    # ROUGE
    rouge_result = rouge.compute(predictions=decoded_preds, references=decoded_labels)

    #COMET
    comet_result = comet.compute(predictions=decoded_preds, references=decoded_labels)



    # Combine all results into a dictionary
    metrics = {
        "bleu": bleu_result["bleu"],
        "chrf": chrf_result["score"],
        "rouge1": rouge_result["rouge1"].fmeasure,
        "rouge2": rouge_result["rouge2"].fmeasure,
        "rougeL": rouge_result["rougeL"].fmeasure,
        "rougeLsum": rouge_result.get("rougeLsum", None),
        "comet": comet_result.get("score", None),
    }

    return metrics

### Training interpretation model

In [24]:
model = model.to(device)

In [25]:
model.name_or_path

'gpt2'

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

# Set training arguments

training_args = TrainingArguments(
    output_dir=f'./results/{model_choice}',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    logging_dir='./logs',
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    weight_decay=0.01,
    learning_rate=9e-5,
    save_strategy="epoch"
)


# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    #compute_metrics=compute_metrics,  # Add compute_metrics if you have it defined
)


In [ ]:
if mode == 'train':
  trainer.train()
  # Save the model
  model.save_pretrained(f'./results/{model_choice}/my_model')
  tokenizer.save_pretrained(f'./results/{model_choice}/my_model')

### Interpretation Evaluation

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(eval_results)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


{'eval_loss': 1.9339402914047241, 'eval_model_preparation_time': 0.0126, 'eval_runtime': 7.3503, 'eval_samples_per_second': 42.583, 'eval_steps_per_second': 5.442}


In [ ]:
def classify_sarcasm(text):
    inputs = classifier_tokenizer(text, return_tensors="pt").to(device)  # Move inputs to the same device as the model
    # Move the model to the same device as the inputs
    classifier_model.to(device)
    outputs = classifier_model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return prediction == 1

In [ ]:
def inference(input_text):
    # Clear prompt for the model
    prompt = "Rewrite this sarcastic comment as a factual statement: "
    if not input_text.startswith(prompt):
        input_text = prompt + input_text

    # Tokenize with padding and attention mask
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True).to(device)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Generate with increased diversity
    output_ids = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_length=64,
        early_stopping=True,
        num_beams=5,              # Beam search for best results
        temperature=0.7,           # Lower temperature for diversity
        top_k=50,                  # Top-k sampling
        pad_token_id=tokenizer.eos_token_id
    )

    # Decode the output, removing the instruction if it is repeated
    decoded_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    if decoded_output.startswith(prompt):
        decoded_output = decoded_output[len(prompt):].strip()

    return decoded_output


In [ ]:
# Inference on a few test examples
evaluation_texts = df_eval.sample(n=10, random_state=42)
count = 0
for index, row in evaluation_texts.iterrows():
    is_sarcastic = classify_sarcasm(row['Sarcastic'])
    print(f"Evaluation {count}:")
    print(f"Original: {row['Sarcastic']}")
    print(f"Sarcastic: {'Yes' if is_sarcastic else 'No'}")
    if row['IsSarcastic'] == 1:
      interpretation = inference(row['Sarcastic'])
      print(f"Interpretation: {interpretation}")
      print(f"Ground Truth: {row['Translation']}")
    print("-" * 20)
    count += 1

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Evaluation 0:
Original: Schadenfreude x
Sarcastic: Yes


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Interpretation: <pad> Schadenfreude x is a bad word. It is a bad word. It is a bad word. It is a bad word. It is a bad word. It is a bad word. It is a bad word. It is a bad word. It is a
Ground Truth: After many years it is brilliant to laugh at germany 
--------------------
Evaluation 1:
Original: So, a country that is trillions in debt, with numbers approaching 13 million on surgical waiting lists and hurtling towards facism regime, is talking about rescuing people from a facist regime?
Sarcastic: Yes
Interpretation: <pad> a country that is trillions in debt, with numbers approaching 13 million on surgical waiting lists and hurtling towards facism regime, is talking about rescuing people from a facist regime?
Ground Truth: This country is not a "free" country anymore.
--------------------
Evaluation 2:
Original: If anyone wants to know how my nights going I tried making a private story on Snapchat and instead made a group chat... I hate my life
Sarcastic: Yes
Interpretation: <pad> I 

In [ ]:
src = "Look at you, finishing all your snacks before dinner. What a healthy choice!"
truth = "Eating snacks before dinner is not a good decision for your health."
print(f"src: {src} \ntranslation: {inference(src)} \nground_truth: {truth}")


src: Look at you, finishing all your snacks before dinner. What a healthy choice! 
translation: <pad> Finishing snacks before dinner is not a good choice for your health. 
ground_truth: Eating snacks before dinner is not a good decision for your health.
